In [7]:
import pandas as pd
import numpy as np

MODEL_TYPE = "SEQ_2_SEQ"

# Data

In [8]:
df = pd.read_csv(f"../../data/processed/20231220_metrics_{MODEL_TYPE}.csv")
df["reference_tokens"] = [
    eval(t)
    for t in df["reference_tokens"]
]
df["generated_tokens"] = [
    eval(t)
    for t in df["generated_tokens"]
]
df["input_tokens"] = [
    eval(t)
    for t in df["input_tokens"]
]
df.head(2)

,input,target,sesgo_pronombre,sesgo_otro,seq2seq_document,causal_document,generation,reference_tokens,max_ref_len,generated_tokens,input_tokens,bleu_gen,bleu_input,bleu_dif,rouge
0,Estimada comunidad beauchefiana: ¿Tienes papel...,['Estimada comunidad beauchefiana: ¿Tienes pap...,NO,NO,Eliminar sesgo de género del siguiente texto:\...,<human>: ¿Puedes reescribir el siguiente texto...,Estimada comunidad beauchefiana:¿Tienes papele...,"[[Estimada, comunidad, beauchefiana, :, ¿Tiene...",11,"[Estimada, comunidad, beauchefiana, :, ¿Tienes...","[Estimada, comunidad, beauchefiana, :, ¿Tienes...",1.0,1.0,0.0,1.0
1,Desde hoy y hasta el 19 de diciembre puedes de...,['Desde hoy y hasta el 19 de diciembre puedes ...,NaN,NaN,Eliminar sesgo de género del siguiente texto:\...,<human>: ¿Puedes reescribir el siguiente texto...,Desde hoy y hasta el 19 de diciembre puedes de...,"[[Desde, hoy, y, hasta, el, 19, de, diciembre,...",17,"[Desde, hoy, y, hasta, el, 19, de, diciembre, ...","[Desde, hoy, y, hasta, el, 19, de, diciembre, ...",1.0,1.0,0.0,1.0


In [9]:
df_with_bias = df[
    (df["sesgo_pronombre"] == "SI") | (df["sesgo_otro"] == "SI") 
]

df_without_bias = df[
    (df["sesgo_pronombre"] == "NO") & (df["sesgo_otro"] == "NO") 
]

df_not_able_to_bias = df[
    (df["sesgo_pronombre"].isna()) & (df["sesgo_otro"].isna()) 
]

print(df.shape, df_with_bias.shape[0] + df_without_bias.shape[0] + df_not_able_to_bias.shape[0])

(782, 15) 782


# Revision

## Has bias

### No diff

In [10]:
wb_0 = df_with_bias[df_with_bias["bleu_dif"] == 0]
print(wb_0.shape)
wb_0["bleu_input"].describe()

(24, 15)


count    24.000000
mean      0.916998
std       0.096434
min       0.594604
25%       0.916653
50%       0.948767
75%       0.974964
max       0.985538
Name: bleu_input, dtype: float64

### Input more similar than generation

In [11]:
wb_below_0 = df_with_bias[df_with_bias["bleu_dif"] < 0]
wb_below_0.shape

(25, 15)

In [12]:
for i, r in wb_below_0.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

50
I: Estimada Comunidad , # InriaChile convoca al ecosistema de investigadores científicos y expertos en ciencias y tecnologías digitales a las Journées Scientifiques Inria Chile 2023 para conocer y debatir sobre los últimos avances en # InteligenciaArtificial🧠 , modelamiento matemático , simulación y optimización , redes y # Iot .
G: Estimada Comunidad , # InriaChile convoca al ecosistema de investigadores científicos y expertos en ciencias y tecnologías digitales a las Journées Scientifiques Inria Chile 2023 para conocer y debatir sobre losúltimos avances en # InteligenciaArtificial , modelamiento matemático , simulación y optimización , redes y # Iot .
{'últimos', 'los', 'InteligenciaArtificial', 'InteligenciaArtificial🧠', 'losúltimos'}

51
I: Únete y serás parte de un gran evento de colaboración franco-chilena🇨🇱🇫🇷 , donde habrá conferencias magistrales de investigadores franceses y chilenos , presentaciones científicas , mesas redondas networking y más .
G: ¡nete y serás parte de 

### Generation more similar than input

In [13]:
wb_above_0 = df_with_bias[df_with_bias["bleu_dif"] > 0]
print(wb_above_0.shape)
wb_above_0["bleu_gen"].describe()

(9, 15)


count    9.0
mean     1.0
std      0.0
min      1.0
25%      1.0
50%      1.0
75%      1.0
max      1.0
Name: bleu_gen, dtype: float64

In [14]:
not_1 = wb_above_0[wb_above_0["bleu_gen"] < 1]

for i, r in not_1.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

## Doesn't have bias

### No diff

In [15]:
nb_0 = df_without_bias[df_without_bias["bleu_dif"] == 0]
print(nb_0.shape)
nb_0["bleu_input"].describe()

(203, 15)


count    203.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: bleu_input, dtype: float64

In [16]:
not_1 = nb_0[nb_0["bleu_input"] < 1]

for i, r in not_1.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

### Input more similar than generation

In [17]:
nb_below_0 = df_without_bias[df_without_bias["bleu_dif"] < 0]
nb_below_0.shape

(68, 15)

In [18]:
for i, r in nb_below_0.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

66
I: Fecha : Miércoles 29 de Noviembre , 12:00-14:30 hrsLugar : Beauchef 851 , piso -3 ( bajo la escalera caracol ) .Ver afiche adjunto para difusión ¡Les esperamos !
G: Fecha : Miércoles 29 de Noviembre , 12:00-14:30 hrsLugar : Beauchef 851 , piso -3 ( bajo la escalera caracol ) .Ver afiche adjunto para difusiónúLes esperamos !
{'¡Les', 'difusión', 'difusiónúLes'}

76
I: Estimad @ s Estudiantes : De acuerdo a la información enviada desde la DirBDE , a partir de las 16.00 hrs de hoy se encontrarán disponibles los resultados de la última asignación 2023 en la plataforma Consulta de Beneficios Los pagos se realizarán el 29 de noviembre en las cuentas informadas en el Formulario de Datos Bancarios .
G: Estimad @ s Estudiantes : De acuerdo a la información enviada desde la DirBDE , a partir de las 16.00 hrs de hoy se encontrarán disponibles los resultados de laúltima asignación 2023 en la plataforma Consulta de Beneficios Los pagos se realizarán el 29 de noviembre en las cuentas informada

### Generation more similar than input

In [19]:
nb_above_0 = df_without_bias[df_without_bias["bleu_dif"] > 0]
print(nb_above_0.shape)
nb_above_0["bleu_gen"].describe()

(0, 15)


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: bleu_gen, dtype: float64

## Unable to bias

### No diff

In [20]:
ub_0 = df_not_able_to_bias[df_not_able_to_bias["bleu_dif"] == 0]
print(ub_0.shape)
ub_0["bleu_input"].describe()

(346, 15)


count    346.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: bleu_input, dtype: float64

In [21]:
not_1 = ub_0[ub_0["bleu_input"] < 1]

for i, r in not_1.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

### Input more similar than generation

In [22]:
ub_below_0 = df_not_able_to_bias[df_not_able_to_bias["bleu_dif"] < 0]
ub_below_0.shape

(107, 15)

In [23]:
for i, r in ub_below_0.iterrows():
    print(i)
    print("I:", " ".join(r['input_tokens']))
    print("G:", " ".join(r['generated_tokens']))
    print(set(r['input_tokens']).symmetric_difference(set(r['generated_tokens'])))
    print()

46
I: ¡Por fin llega nuevamente al Teatro Nacional Chileno la 4ta edición del Encuentro de Teatro Interfacultades de la Universidad de Chile , organizado por la Biblioteca Central de la FCFM !
G: ¿Por fin llega nuevamente al Teatro Nacional Chileno la 4ta edición del Encuentro de Teatro Interfacultades de la Universidad de Chile , organizado por la Biblioteca Central de la FCFM !
{'¡Por', '¿Por'}

52
I: ¡No te lo pierdas !
G: ¿No te lo pierdas !
{'¿No', '¡No'}

54
I: Inscríbete y conoce los detalles del programa de las Journées Scientifiques de Inria Chile 2023 👉 https : //journeesscientifiques2023.inria.cl/El programa completo está disponible acá , incluyendo el del día en la Facultad que será el martes 05 de diciembre .
G: Inscríbete y conoce los detalles del programa de las Journées Scientifiques de Inria Chile 2023 https : //journeesscientifiques2023.inria.cl/El programa completo está disponible acá , incluyendo el del día en la Facultad que será el martes 05 de diciembre .
{'👉'}



### Generation more similar than input

In [24]:
ub_above_0 = df_not_able_to_bias[df_not_able_to_bias["bleu_dif"] > 0]
print(ub_above_0.shape)
ub_above_0["bleu_gen"].describe()

(0, 15)


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: bleu_gen, dtype: float64